In [4]:
import numpy as np
import cv2
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

def detect_features_and_form_clusters(input_image, num_clusters=10, feature_type='SIFT'):
    # Choose feature detector
    feature_detectors = {
        'SIFT': cv2.SIFT_create,
        'AKAZE': cv2.AKAZE_create
    }
    
    if feature_type in feature_detectors:
        feature_detector = feature_detectors[feature_type]()
    else:
        raise ValueError("Feature detector not recognized.")
    
    # Find keypoints and their descriptors
    keypoints, feature_descriptors = feature_detector.detectAndCompute(input_image, None)
    
    # Handle case with no keypoints found
    if feature_descriptors is None:
        return np.array([]), np.array([])

    # Clustering using KMeans
    kmeans_cluster = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans_cluster.fit(feature_descriptors)
    
    # Get the feature positions
    feature_positions = np.array([keypoint.pt for keypoint in keypoints], dtype=np.float32)
    
    # Assign clusters
    assigned_clusters = kmeans_cluster.labels_
    
    return feature_positions, assigned_clusters

def create_spatial_hist_of_features(image, feature_positions, assigned_clusters, grid_divisions):
    # Define grid divisions
    grid_rows, grid_cols = grid_divisions
    grid_height, grid_width = image.shape[:2]
    
    # Histogram initialization
    cluster_histogram = np.zeros(np.unique(assigned_clusters).size, dtype=int)
    spatial_hist = []
    
    # Compute histogram for each grid section
    for row in range(grid_rows):
        for col in range(grid_cols):
            # Grid section boundaries
            row_start, col_start = row * (grid_height // grid_rows), col * (grid_width // grid_cols)
            row_end, col_end = row_start + (grid_height // grid_rows), col_start + (grid_width // grid_cols)
            
            # Build histogram for current grid section
            grid_hist = np.zeros_like(cluster_histogram)
            for position, cluster in zip(feature_positions, assigned_clusters):
                y, x = position
                if row_start <= y < row_end and col_start <= x < col_end:
                    grid_hist[cluster] += 1
            
            spatial_hist.extend(grid_hist)
    
    return np.array(spatial_hist)



image_path = r"C:\Users\risha\Desktop\2nd term\Computer vision\assignment\CA_2024\CA_2024\data\notre_dame_1.jpg"
image = cv2.imread(image_path)

# Ensure the image was loaded correctly
if image is None:
    raise ValueError("Image not found or unable to load.")

# Detect features and form clusters
feature_positions, assigned_clusters = detect_features_and_form_clusters(image)

division_patterns = [[2, 2], [2, 3], [1, 1]]
histograms = []

for pattern in division_patterns:
    histogram = create_spatial_hist_of_features(image, feature_positions, assigned_clusters, pattern)
    histograms.append(histogram)

    # Save the histogram to a CSV file
    csv_filename = f"./histogram_division_{pattern[0]}x{pattern[1]}.csv"
    np.savetxt(csv_filename, histogram.reshape(1, -1), delimiter=",", fmt="%d")
    print(f"Histogram for division {pattern} saved to '{csv_filename}'")



C:\Users\risha\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Histogram for division [2, 2] saved to './histogram_division_2x2.csv'
Histogram for division [2, 3] saved to './histogram_division_2x3.csv'
Histogram for division [1, 1] saved to './histogram_division_1x1.csv'
